In [1]:
import os
import pathlib

import dotenv
import numpy as np
import xarray as xr
import xroms

# Filepaths

In the project code directory there is a `.env` file that defines various filepaths. For example, one line is `SANTA_BARBARA_BASIN_MODEL_OUTPUT=${GROUP_SCRATCH}/Santa_Barbara_basin_model_output`. We can load the values from this file using the `dotenv` package. Passing `interpolate=True` tells `dotenv_values` to interpolate the value of environment variables such as `GROUP_SCRATCH` so that we get the full path.

Then we use `pathlib` to extract all the ROMS average files and the grid file.

In [2]:
# get paths
ENV_VARIABLES = dotenv.dotenv_values(interpolate=True)
MODEL_OUTPUT_DIR = ENV_VARIABLES['SANTA_BARBARA_BASIN_MODEL_OUTPUT']
MODEL_OUTPUT_DIR

'/scratch/groups/leift/Santa_Barbara_basin_model_output'

In [3]:
# get roms filepaths
AVG_FILES = sorted(pathlib.Path(MODEL_OUTPUT_DIR).glob("*_avg.*.nc"))
GRID_FILE = next(pathlib.Path(MODEL_OUTPUT_DIR).glob("*_grd.nc"), None)
GRID_FILE

PosixPath('/scratch/groups/leift/Santa_Barbara_basin_model_output/usw3_grd.nc')

# Opening the dataset

## The time dimension

Different versions of ROMS use different conventions for the time dimension. The dataset we're opening here has a `time` dimension and defines an `ocean_time` variable that stores the time in seconds since initialisation. The analysis software we use, `xroms`, expects an `ocean_time` dimension. Moreover, in the datasets, the `time` dimension has no coordinate attached to it and consequently we cannot immediately concatenate the output files along the time dimension.

Fortunately, this can be solved with a simple one line function. We then open the files as a multi-file dataset using `xr.open_mfdataset`, passing the preprocessing function.

## The grid

`xroms` also requires various grid quantities. Some versions of ROMS save these in the output files but here we have to add them explicitly. This is also straight-forward. First, we open the grid file and then we merge the two datasets. 

[Roms wiki](https://www.myroms.org/wiki/Vertical_S-coordinate)

In [4]:
def make_ocean_time_dim(ds: xr.Dataset) -> xr.Dataset:
    """Make ocean time the dimension."""
    return ds.swap_dims(time="ocean_time")

# open avg files as one dataset
ds = xr.open_mfdataset(AVG_FILES, preprocess=make_ocean_time_dim)


In [5]:
def add_s_coordinates(ds: xr.Dataset) -> xr.Dataset:
    """Construct and add the s-coordinates."""
    ds["Cs_w"] = ("s_w", ds.attrs["Cs_w"])
    ds["Cs_r"] = ("s_rho", ds.attrs["Cs_r"])
    ds["s_w"] = (s_w := np.linspace(-1,0,ds.Cs_w.size, endpoint=True))
    ds["s_rho"] = s_w[:-1] + np.diff(s_w) / 2
    return ds

ds = add_s_coordinates(ds)
grd = xr.open_dataset(GRID_FILE)
ds = ds.merge(grd)
ds, xgrid = xroms.roms_dataset(ds, Vtransform=2)
ds

<xarray.Dataset> Size: 19TB
Dimensions:     (ocean_time: 1703, eta_rho: 852, xi_rho: 1702, xi_u: 1701,
                 eta_v: 851, s_rho: 50, s_w: 51)
Coordinates: (12/17)
  * ocean_time  (ocean_time) float64 14kB 0.0 0.0 0.0 ... 4.118e+08 4.118e+08
  * s_w         (s_w) float64 408B -1.0 -0.98 -0.96 -0.94 ... -0.04 -0.02 0.0
  * s_rho       (s_rho) float64 400B -0.99 -0.97 -0.95 ... -0.05 -0.03 -0.01
    lon_psi     (eta_v, xi_u) float64 12MB ...
    lat_psi     (eta_v, xi_u) float64 12MB ...
  * xi_rho      (xi_rho) int64 14kB 0 1 2 3 4 5 ... 1697 1698 1699 1700 1701
    ...          ...
    z_w_v       (ocean_time, s_w, eta_v, xi_rho) float64 1TB dask.array<chunksize=(1, 51, 426, 851), meta=np.ndarray>
    z_w_psi     (ocean_time, s_w, eta_v, xi_u) float64 1TB dask.array<chunksize=(1, 51, 426, 851), meta=np.ndarray>
    z_rho       (ocean_time, s_rho, eta_rho, xi_rho) float64 988GB dask.array<chunksize=(1, 50, 426, 851), meta=np.ndarray>
    z_rho_u     (ocean_time, s_rho, eta_rho, xi_u) float64 987GB dask.array<chunksize=(1, 50, 426, 851), meta=np.ndarray>
    z_rho_v     (ocean_time, s_rho, eta_v, xi_rho) float64 987GB dask.array<chunksize=(1, 50, 426, 851), meta=np.ndarray>
    z_rho_psi   (ocean_time, s_rho, eta_v, xi_u) float64 986GB dask.array<chunksize=(1, 50, 426, 851), meta=np.ndarray>
Data variables: (12/49)
    zeta        (ocean_time, eta_rho, xi_rho) float32 10GB dask.array<chunksize=(1, 426, 851), meta=np.ndarray>
    ubar        (ocean_time, eta_rho, xi_u) float32 10GB dask.array<chunksize=(1, 426, 851), meta=np.ndarray>
    vbar        (ocean_time, eta_v, xi_rho) float32 10GB dask.array<chunksize=(1, 426, 851), meta=np.ndarray>
    u           (ocean_time, s_rho, eta_rho, xi_u) float32 494GB dask.array<chunksize=(1, 10, 171, 341), meta=np.ndarray>
    v           (ocean_time, s_rho, eta_v, xi_rho) float32 493GB dask.array<chunksize=(1, 10, 171, 341), meta=np.ndarray>
    temp        (ocean_time, s_rho, eta_rho, xi_rho) float32 494GB dask.array<chunksize=(1, 10, 171, 341), meta=np.ndarray>
    ...          ...
    dz_v        (ocean_time, s_rho, eta_v, xi_rho) float64 987GB dask.array<chunksize=(1, 50, 426, 851), meta=np.ndarray>
    dz_w_v      (ocean_time, s_w, eta_v, xi_rho) float64 1TB dask.array<chunksize=(1, 51, 426, 851), meta=np.ndarray>
    dz_psi      (ocean_time, s_rho, eta_v, xi_u) float64 986GB dask.array<chunksize=(1, 50, 426, 851), meta=np.ndarray>
    dz_w_psi    (ocean_time, s_w, eta_v, xi_u) float64 1TB dask.array<chunksize=(1, 51, 426, 851), meta=np.ndarray>
    dA          (eta_rho, xi_rho) float64 12MB 1.512e+04 1.512e+04 ... 1.37e+04
    rho0        int64 8B 1025
Attributes: (12/33)
    type:           ROMS averages file
    title:          L3 MidCal 100 m 50 levels (no wave)
    grid_file:      usw3_grd.000.nc
    init_file:      usw3_rst.000.nc
    ntimes:         1209600
    ndtfast:        50
    ...             ...
    rdrg2_units:    nondimensional
    Zob:            0.01
    Zob_units:      meter
    SRCS:           KRNSRC main.F step2D_FB.F read_inp.F read_sta_pos.F timer...
    CPPS:           <cppdefs.h> USWC SOLVE3D UV_COR UV_ADV CURVGRID SPHERICAL...
    nc_format:      netCDF-4, zlib-compressed

In [6]:
# ds = ds.merge(grd)
# ds["Cs_w"] = ("s_w", ds.attrs["Cs_w"])
# ds["Cs_r"] = ("s_rho", ds.attrs["Cs_r"])
# ds, xgrid = xroms.roms_dataset(ds)
# ds